<a href="https://colab.research.google.com/github/yncute/optcg_scraper/blob/main/optcgScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OPTCG Card Scraping
Goal: scrape all exisitng EN cards from various sites. This includes promos, tournament-exclusive cards, all DON!! cards, Bandai limited collection cards, etc.

#1. Imports and packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install Selenium and WebDriver
!apt-get update
!apt install -y chromium-chromedriver
!pip install selenium

# Set environment variables for Chrome
import os
os.environ['PATH'] += ':/usr/lib/chromium-browser/chromedriver'

#imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Keys, ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import requests
import time
import csv
import re


# Configure Selenium to use headless Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')  # Runs Chrome in headless mode (no GUI)
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
chrome_options.add_argument('--window-size=1920x1080')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,452 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packa

#2. Formatting and CSV writing functions
Formatting for data scraped from OPTCGPlayer.

In [ ]:
import csv
import re

headers = ['Image', 'Name', 'ID', 'Color', 'Card Category', 'Rarity',
           'Alt Art', 'Price', 'Life','Cost', 'Power', 'Counter', 'Attribute', 'Type',
           'Card Set', 'Effect', 'Trigger']
csv_path = '/content/drive/MyDrive/dataset/op_cards.csv'

# extracts full effect description excluding trigger
def extract_effect(lines, start_index):
    """Extracts all lines after 'Effect' until '[Trigger]' or end of lines."""
    effect_lines = []
    i = start_index + 1
    while i < len(lines) and '[Trigger]' not in lines[i]:
        effect_lines.append(lines[i].strip())
        i += 1
    return " ".join(effect_lines), i  # Returns the effect text and the next index



def format_data(raw):
  lines = raw.strip().split("\n")
  print(lines)

  card_data = {
      'Image': re.search(r'img source: (.*)', lines[0]).group(1) if "img source:" in lines[0] else '',
      'Name': lines[1].strip(),
      'ID': lines[2].strip(),
      'Color': lines[3].strip(),
      'Card Category': lines[4].strip(),
      'Rarity': lines[5].strip(),
      'Alt Art': "Alt Art" if 'Alt Art' in lines else '',
      'Price': next((line.split("Price: ")[1] for line in lines if "Price: " in line), ""),
      **{line: (lines[i + 1].strip() if i + 1 < len(lines) else '') for i, line in enumerate(lines) if line in ['Life', 'Cost', 'Power', 'Counter', 'Attribute', 'Type', 'Card Set', 'Effect', 'Trigger']}
  }

  #print(card_data)
  return card_data


# needs fixing


# def format_data(raw):
#     lines = raw.strip().split("\n")

#     # Handle simple key-value pairs using dictionary comprehension
#     basic_fields = {
#         line: (lines[i + 1].strip() if i + 1 < len(lines) else '')
#         for i, line in enumerate(lines)
#         if line in ['Life', 'Cost', 'Power', 'Counter', 'Attribute', 'Type', 'Card Set']
#     }

#     # Handle multi-line fields like 'Effect' and other complex fields
#     effect_text, trigger_text = '', ''
#     for i, line in enumerate(lines):
#         if line == 'Effect':
#             effect_text, next_index = extract_effect(lines, i)
#         elif line == '[Trigger]':
#             trigger_text = lines[i + 1].strip() if i + 1 < len(lines) else ''

#     # Combine all fields into the final dictionary
#     card_data = {
#         'Image': re.search(r'img source: (.*)', lines[0]).group(1) if "img source:" in lines[0] else '',
#         'Name': lines[1].strip(),
#         'ID': lines[2].strip(),
#         'Color': lines[3].strip(),
#         'Card Category': lines[4].strip(),
#         'Rarity': lines[5].strip(),
#         'Alt Art': "Alt Art" if 'Alt Art' in lines else '',
#         'Price': next((line.split("Price: ")[1] for line in lines if "Price: " in line), ""),
#         **basic_fields,
#         'Effect': effect_text,
#         'Trigger': trigger_text,
#     }



#writes a row of data to the csv with specified headers
def save_to_csv(data, path):
  with open(path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data)

# Specify the row indices (array) you want to delete (0-based index) from csv at path
def del_from_csv(path, row_indices):
  with open(path, mode="r", newline="") as infile:
      reader = csv.reader(infile)
      rows = [row for idx, row in enumerate(reader) if idx not in row_indices]

  with open(path, mode="w", newline="") as outfile:
      writer = csv.writer(outfile)
      writer.writerows(rows)
  print(f"Rows at indices {row_indices} have been deleted.")


##testing

In [ ]:
cards_list = []
test_1 = """
img source: https://optcgplayer.com/images/EN/DON-DCP1_p4.png
Don
DON-DCP1
-
DON
D
Alt Art
Price: $71.11
Cost
0
Power
0
Counter
0
Attribute
-
Type
-
Card Set
DON!! Card Pack Vol. 1
Effect
-
"""

test_2 = """
img source: https://optcgplayer.com/images/EN/EB01-013_p1.png
Kouzuki Hiyori
EB01-013
Green
CHARACTER
R
Alt Art
Price: $14.23
Cost
4
Power
0
Counter
1000
Attribute
Wisdom
Type
Land of Wano/Kouzuki Clan
Card Set
EB01
Effect
Activate: Main You may trash this Character: Play up to 1 {Land of Wano} type Character card with a cost of 5 or less other than [Kouzuki Hiyori] from your hand. Then, draw 1 card.
"""

test_trigger = """
img source: https://optcgplayer.com/images/EN/ST05-016.png
Lion's Threat Imperial Earth Bind
ST05-016
Purple
EVENT
C
Price: $0.58
Cost
3
Counter
0
Type
FILM/Golden Lion Pirates
Card Set
ST05
Effect
Main DON!! −2 (You may return the specified number of DON!! cards from your field to your DON!! deck.): K.O. up to 1 of your opponent's Characters with a cost of 5 or less.
Trigger
[Trigger] Add up to 1 DON!! card from your DON!! deck and set it as active.
"""

test_trigger2 = """
img source: https://optcgplayer.com/images/EN/ST07-004.png
Charlotte Snack
ST07-004
Yellow
CHARACTER
C
Price: $0.07
Cost
5
Power
6000
Counter
0
Attribute
Slash
Type
Big Mom Pirates
Card Set
ST07
Effect
DON!! x1 When Attacking You may add 1 card from the top or bottom of your Life cards to your hand: This Character gains Banish and +1000 power during this battle.
(When this card deals damage, the target card is trashed without activating its Trigger.)
"""

test_life = """
img source: https://optcgplayer.com/images/EN/ST06-001.png
Sakazuki
ST06-001
Black
LEADER
L
Price: $0.75
Life
5
Power
5000
Counter
0
Attribute
Special
Type
Navy
Card Set
ST06
Effect
Activate: Main Once Per Turn ③ (You may rest the specified number of DON!! cards in your cost area.) You may trash 1 card from your hand: K.O. up to 1 of your opponent's Characters with a cost of 0.
"""

test_multiline_effect = """
img source: https://optcgplayer.com/images/EN/ST20-005.png
Charlotte Linlin
ST20-005
Yellow
CHARACTER
SR
Cost
6
Power
7000
Counter
0
Attribute
Special
Type
The Four Emperors/Big Mom Pirates
Card Set
ST20
Effect
On Play You may trash 1 card from your hand: Your opponent chooses one:
• Your opponent trashes 2 cards from their hand.
• Trash 1 card from the top of your opponent's Life cards.
"""

cards_list.append(format_data(test_1))
cards_list.append(format_data(test_2))
cards_list.append(format_data(test_life))
cards_list.append(format_data(test_trigger))
cards_list.append(format_data(test_trigger2))
cards_list.append(format_data(test_multiline_effect))
save_to_csv(cards_list, '/content/testing.csv')

AttributeError: 'NoneType' object has no attribute 'keys'

#3. Scraping from https://onepiece-cardgame.dev/cards

First scraping attempt. Source is easy to scrape from, however lacks several hundred cards

In [ ]:


# Configure Selenium to use headless Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')  # Runs Chrome in headless mode (no GUI)
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
chrome_options.add_argument('--window-size=1920x1080')

# Start the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Load the website
url = 'https://onepiece-cardgame.dev/cards'
driver.get(url)

# Wait for the page to load (adjust time depending on how long it takes)
time.sleep(5)

# Find the image elements (adjust selector if needed)
div_class = "MuiBox-root.css-1xdp8rh"
cards = driver.find_elements(By.CSS_SELECTOR, f'div.{div_class}')

# directory in Google Drive to save the images
drive_folder = '/content/drive/MyDrive/dataset/optcg_cards'

# dict to store each card info
cards_list = []

popup_id = 'simple-popper'

for card in cards[:0]:

    #click on card to prompt tooltip
    driver.execute_script("arguments[0].click();", card)
    #wait for tooltip to load
    WebDriverWait(driver, 5)

    popup_text_parent = driver.find_element(By.XPATH, "//*[@id='simple-popper']/div/div/div[2]/div[2]")
    child_text_divs = popup_text_parent.find_elements(By.XPATH, "./div[position() > 1]//div")

    child_image_src = driver.find_element(By.XPATH, "//*[@id='contextImage']")
    image_url = child_image_src.get_attribute('src')

    #initialize card with image src field
    new_card = {
        "img_src":image_url
    }

    #append key/value pairs to the card data
    for i in range(0, len(child_text_divs), 2):
      try:
        # Get the key from the even-indexed div
        key = child_text_divs[i].text.strip()

        # Get the value from the odd-indexed div
        value = child_text_divs[i+1].text.strip()

        # Append the dictionary to the cards list
        new_card[key] = value
      except IndexError:
        # In case there's an uneven number of divs
        print(f"Skipping div at index {i}, no matching value.")
        continue

    cards_list.append(new_card)

# Close the browser after scraping
driver.quit()

print(f"Scraping complete!")


Scraping complete!


#4. Scraping from TCGPlayer
Second attempt. Source has every EN card, however difficult to scrape/takes a very long time

##4.1 Defining Methods

Process

1.   Collect a list of urls for all listings from specified root
*   Define rules for what listings to fetch

2.   Iterate through list and scrape



In [ ]:
# Configure Selenium to use headless Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')  # Runs Chrome in headless mode (no GUI)
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
chrome_options.add_argument('--window-size=1920x1080')

# Start the WebDriver
driver = webdriver.Chrome(options=chrome_options)

## __ HELPER FUNCTIONS __

#check for last page of listing
def is_last_page(driver):
    try:
        # if next button is disabled/enabled
        next_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next page"]')
        if next_btn.get_attribute("disabled") == "true" and next_btn.get_attribute("aria-disabled") == "true":
          print("reached last page")
          return True
        else:
          return False

    except NoSuchElementException:
        # If no "Next" button is found, assume we're on the last page
        print("no next btn found. assuming this is the last page")
        return True

#check if listing is presale
def is_presale(driver, listing):
  try:
    #check for a presale badge on the listing
    listing_badge = listing.find_element(By.CSS_SELECTOR, ".mp-presale-badge__inline")
    print("presale badge found:", listing_badge.text)
    return True
  except NoSuchElementException:
    return False
  return False

# scrapes individual listing from page and returns a card object with its data
def scrape_listing(driver, listing_url):
  new_card = {}

  #load the url
  driver.get(listing_url)

  wait = WebDriverWait(driver, 15)

  try:
    img_element = wait.until(
      EC.visibility_of_element_located((By.XPATH, "//*[@id='app']/div/div/section[2]/section/div[2]/div[2]/section[1]/section/div/div/div/div/div/div/img"))
    )
  except Exception as e:
    print(f"Exception: {e}")

  img_srcset = img_element.get_attribute("srcset")
  # img_srcset has many image urls, we only want the highest res (aka last one)
  new_card["img_url"] = img_srcset.split(',')[-1].strip().split(' ')[0]
  print(new_card['img_url'])

  try:
    effect = driver.find_element(By.CSS_SELECTOR, "div.product__item-details__description")
    new_card["Effect"] = effect.text
  except Exception as e:
    print(f"Exception: {e}")

  try:
    name = driver.find_element(By.CSS_SELECTOR, "h1.product-details__name")
    new_card["listing_name"] = name.text
  except Exception as e:
    print(f"Exception: {e}")

  try:
    attributes_list = driver.find_element(By.CSS_SELECTOR, "ul.product__item-details__attributes")
    att_list_items = attributes_list.find_elements(By.XPATH, ".//li")
    for item in att_list_items:
      if(item.text):
        key, value = item.text.split(":",1)
        new_card[key.strip()] = value.strip()
  except Exception as e:
    print(f"Exception: {e}")

  return new_card

# __MAIN__

def scrape_from_url(l,r):
  ## __VARIABLES__
  tcg_cards_list = []
  listing_urls = []
  page_num = 1

  while True:
    url = l + str(page_num) + r
    print("Scraping from: ", url)
    driver.get(url)

    #wait for page to load
    wait = WebDriverWait(driver, 10)

    #Get all listing URLs
    listings_container = wait.until(
      EC.visibility_of_element_located((By.CSS_SELECTOR, "section.search-results"))
  )
    listings = listings_container.find_elements(By.XPATH, "./*")
    print(f"page {page_num}: found {len(listings)} listings")

    #iterate through listings
    for item in listings:

      #don't scrape if it's a presale (bc it has no info)
      if not is_presale(driver, item):
        listing_href_element = item.find_element(By.CSS_SELECTOR, "a[href]")
        listing_url = listing_href_element.get_attribute('href')
        listing_urls.append(listing_url)

    #break out
    if len(listings) == 0:
      print("no remaining listings")
      break
    if is_last_page(driver):
      break

    page_num += 1

  print(f"Found {len(listing_urls)} listings: {listing_urls}")

  # scrape from list of urls
  count = 1
  for item in listing_urls:
    tcg_cards_list.append(scrape_listing(driver, item))
    print(f"{count} scraping listing at url {item}")

    count += 1

  return tcg_cards_list




In [ ]:
driver = webdriver.Chrome(options=chrome_options)
def scrape_price_from_tcgplayer(url):
  prices = {
      "market_price":"",
      "last_sold":"",
  }
  driver.get(url)
  driver.maximize_window()
  wait = WebDriverWait(driver, 5)

  try:
    prices_div = wait.until(
      EC.visibility_of_element_located((By.CLASS_NAME, "price-points__upper"))
    )
    market = prices_div.find_element(By.XPATH, "./table/tr[1]/td[2]/span")
    last = prices_div.find_element(By.XPATH, "./table/tr[2]/td[2]/span")

    prices["market_price"] = market.text
    prices["last_sold"] = last.text

  except Exception as e:
    print(f"Exception: {e}")
  return prices

scrape_price_from_tcgplayer("https://www.tcgplayer.com/product/558101/one-piece-card-game-two-legends-charlotte-linlin?page=1&Language=English")

{'market_price': '$3.02', 'last_sold': '$2.50'}

In [ ]:
#test helpers

list_url = "https://www.tcgplayer.com/product/558137/one-piece-card-game-two-legends-kalgara-099?page=1&Language=English"
scrape_listing(driver, list_url)

In [ ]:
import csv

#ALL CARDS ROOT URL
#https://www.tcgplayer.com/search/one-piece-card-game/product?productLineName=one-piece-card-game&page=1&view=grid&ProductTypeName=Cards

url_all_l = "https://www.tcgplayer.com/search/one-piece-card-game/product?productLineName=one-piece-card-game&page="
url_all_r = "&view=grid&ProductTypeName=Cards"

#op08 cards
url_08_l = "https://www.tcgplayer.com/search/one-piece-card-game/two-legends?productLineName=one-piece-card-game&page="
url_08_r = "&view=grid&ProductTypeName=Cards&setName=two-legends"

#don cards
url_don_l = "https://www.tcgplayer.com/search/one-piece-card-game/product?productLineName=one-piece-card-game&page="
url_don_r = "&view=grid&CardType=DON!!"

#will not include presales
tcg_cards_list = scrape_from_url(url_don_l, url_don_r)
print(tcg_cards_list)

fieldnames = [
    "listing_name",
    "img_url",
    "Effect",
    "Rarity",
    "Number",
    "Color",
    "Card Type",
    "Life",
    "Cost",
    "Power",
    "Subtype(s)",
    "Counter+",
    "Attribute",
    "Artist"
]
with open('tcg_all_cards.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write the header row (column names)
    writer.writeheader()
    for card in tcg_cards_list:
      writer.writerow(card)

Scraping from:  https://www.tcgplayer.com/search/one-piece-card-game/product?productLineName=one-piece-card-game&page=1&view=grid&CardType=DON!!
page 1: found 24 listings
Scraping from:  https://www.tcgplayer.com/search/one-piece-card-game/product?productLineName=one-piece-card-game&page=2&view=grid&CardType=DON!!
page 2: found 24 listings
Scraping from:  https://www.tcgplayer.com/search/one-piece-card-game/product?productLineName=one-piece-card-game&page=3&view=grid&CardType=DON!!
page 3: found 24 listings
Scraping from:  https://www.tcgplayer.com/search/one-piece-card-game/product?productLineName=one-piece-card-game&page=4&view=grid&CardType=DON!!
page 4: found 24 listings
Scraping from:  https://www.tcgplayer.com/search/one-piece-card-game/product?productLineName=one-piece-card-game&page=5&view=grid&CardType=DON!!
page 5: found 10 listings
reached last page
Found 106 listings: ['https://www.tcgplayer.com/product/593826/one-piece-card-game-premium-booster-the-best-don-card-uta?page=1

###Normalizing TCGPlayer Data to fit OPTCGPlayer CSV

In [ ]:
import pandas as pd
import numpy as np
import math

headers_optcgplayer = ['Image', 'Name', 'ID', 'Color', 'Card Category', 'Rarity',
           'Alt Art', 'Price', 'Life','Cost', 'Power', 'Counter', 'Attribute', 'Type',
           'Card Set', 'Effect', 'Trigger']

fieldnames_tcgplayer = [
    "listing_name",
    "img_url",
    "Effect",
    "Rarity",
    "Number",
    "Color",
    "Card Type",
    "Life",
    "Cost",
    "Power",
    "Subtype(s)",
    "Counter+",
    "Attribute",
    "Artist"
]

dons_csv_path = '/content/drive/MyDrive/dataset/tcg_dons.csv'
test_backup_csv_optcgplayer_path = '/content/drive/MyDrive/dataset/backup_10-31_op_cards.csv'

to_convert = {
    "listing_name":"Name",
    "img_url":"Image",
    "Number":"ID",
    "Card Type":"Card Category",
    "Subtype(s)": "Type",
    "Counter+":"Counter",
}

# returns a list of dictionaries conforming to the new_headers format
# header_conversions is a dict defining what the headers in csv should be converted to
def normalize_fields(header_conversions, new_headers, csv):
  df = pd.read_csv(csv)

  new_list = []

  #convert data to a new csv with
  for index, row in df.iterrows():
    row_dict = row.to_dict()
    cleaned_row = {k:("" if isinstance(v, float) and math.isnan(v) else v) for k, v in row_dict.items()}
    template_dict = {key: "" for key in new_headers}

    for key in cleaned_row.keys():
      if key in header_conversions.keys():
        template_dict[header_conversions[key]] = cleaned_row[key]
    new_list.append(template_dict)

  return new_list

formatted_list = normalize_fields(to_convert, headers_optcgplayer, dons_csv_path)



In [ ]:
with open('test_dons_convert.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers_optcgplayer)

    # Write the header row (column names)
    writer.writeheader()
    for card in formatted_list:
      writer.writerow(card)

#5. Scraping from OPTCGPlayer
Third attempt. Middle ground between the previous two, is not updated as regularly/quickly as TCGPlayer, but easier and faster to scrape from. Lacks many DON cards

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://optcgplayer.com/")
driver.maximize_window()

formatted_cards_list = []

SCROLL_PAUSE_TIME = 1.5  # Time to wait after each scroll

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for the page to load more content
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height after scrolling
    new_height = driver.execute_script("return document.body.scrollHeight")

    # If the scroll height hasn't changed, we've reached the bottom
    if new_height == last_height:
        break

    last_height = new_height

actions = ActionChains(driver)

try:
  cards = driver.find_elements(By.TAG_NAME, "app-card")
  print(len(cards), " cards")
  count = 1

  for card in cards:
    actions.move_to_element(card).click().perform()
    time.sleep(1)

    # Extract data
    data = driver.find_element(By.XPATH,'//*[@id="container"]/div/div[2]/div')
    data_text = data.text

    # Extract img
    image_src = data.find_element(By.CSS_SELECTOR, "img").get_attribute("src")

    raw = "img source: " + image_src + "\n" + data_text
    print(f"{count}/{len(cards)}")
    print(raw, "\n_______________________________________________________")
    formatted_cards_list.append(format_data(raw))

    #click out the modal
    exit_btn = driver.find_element(By.XPATH,"//*[@id='container']/div/div[1]/button")
    exit_btn.click()
    time.sleep(1)

    count += 1

except Exception as ex:
  print(ex)

driver.quit()




Streaming output truncated to the last 5000 lines.
Cost
3
Power
4000
Counter
2000
Attribute
Slash
Type
Navy
Card Set
ST06
Effect
Activate: Main You may rest this Character: Give up to 1 of your opponent's Characters −2 cost during this turn. 
_______________________________________________________
['img source: https://optcgplayer.com/images/EN/ST06-006.png', 'Tashigi', 'ST06-006', 'Black', 'CHARACTER', 'C', 'Price: $2.75', 'Cost', '3', 'Power', '4000', 'Counter', '2000', 'Attribute', 'Slash', 'Type', 'Navy', 'Card Set', 'ST06', 'Effect', "Activate: Main You may rest this Character: Give up to 1 of your opponent's Characters −2 cost during this turn."]
2250/2457
img source: https://optcgplayer.com/images/EN/ST06-006_p2.png
Tashigi
ST06-006
Black
CHARACTER
SP
Alt Art
Price: $43.53
Cost
3
Power
4000
Counter
2000
Attribute
Slash
Type
Navy
Card Set
OP08
Effect
Activate: Main You may rest this Character: Give up to 1 of your opponent's Characters −2 cost during this turn. 
_________________

In [ ]:
save_to_csv(formatted_cards_list, '/content/drive/MyDrive/dataset/op_cards_11_21.csv')